In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score

In [10]:
abalone = fetch_ucirepo(id=1) 
X = abalone.data.features 
y = abalone.data.targets 

In [11]:
X['Sex'].replace('M', 0, inplace = True)
X['Sex'].replace('F', 1, inplace = True)
X['Sex'].replace('I', 2, inplace = True)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

C:\Users\legion\AppData\Local\Temp\ipykernel_12876\919359157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'].replace('M', 0, inplace = True)
C:\Users\legion\AppData\Local\Temp\ipykernel_12876\919359157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'].replace('F', 1, inplace = True)
C:\Users\legion\AppData\Local\Temp\ipykernel_12876\919359157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'].replace('

In [12]:
Xs_train, Xs_test, y_train, y_test = train_test_split(X, y, test_size=1/5, random_state=1)

param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4]
}
# kf = KFold(n_splits=10, shuffle=True, random_state=1)
inner_cv = KFold(n_splits=10, shuffle=True, random_state=1)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=inner_cv, scoring='neg_mean_squared_error')

grid_search.fit(Xs_train, y_train)
non_nested_scores = grid_search.best_score_

nested_cv_results = cross_val_score(grid_search, Xs_train, y_train, cv=outer_cv).mean()

best_params = grid_search.best_params_
best_tree = grid_search.best_estimator_

In [13]:
non_nested_scores

-5.558570512526694

In [14]:
nested_cv_results

-5.624584490076972

In [15]:
# best_tree = DecisionTreeRegressor(max_depth=6)
# best_tree.fit(Xs_train,y_train)

test_mse = mean_squared_error(y_test, best_tree.predict(Xs_test))
test_mse

5.401976481793877

In [16]:
y_test['pred'] = best_tree.predict(Xs_test)
y_test['mape'] = (y_test['pred'] - y_test['Rings']).abs()/y_test['Rings']
y_test.mean()

Rings    9.936603
pred     9.860153
mape     0.158021
dtype: float64